## Retrieveing Data

In [114]:
import requests
import pandas as pd
import time
import json

CURRENT_TIME = round(time.time() * 1000) 

def dateToEpoch(date):
    a = date.split('/')
    timestamp = datetime(int('20' + a[2]),int(a[1]),int(a[0]),0,0).timestamp()
    return int(timestamp * 1000)

def epochToDate(epoch):
    # from datetime import datetime
    # print(datetime.fromtimestamp(int("1518308894652")/1000))
    return time.strftime('%Y-%m-%d %H:%M',time.localtime(epoch/1000))

def jsonToDataFrame(json_resp):
    res = json.loads(json_resp)
    df = pd.DataFrame(res['result'])
    return df

def getSymbols():
    json_resp2 = (requests.get("https://fapi.binance.com/fapi/v1/exchangeInfo").json())
    for j in range(len(json_resp2['symbols'])):

        print(json_resp2['symbols'][j]['symbol'])

def getKlines(pair,interval,startime,endtime=CURRENT_TIME):
    query = { 
    "symbol":pair, 
    "interval": interval,
    "startTime": startime,
    "endTime": endtime
    
    }
    # Add option for endtime
    
    json_resp = (requests.get("https://fapi.binance.com/fapi/v1/klines",params=query).json())
    df = pd.DataFrame(json_resp)
    return df


## Cleaning Data

In [115]:
def dataCleaning(df):
    columns = {
        0:'time',
        1:'open',
        2:'high',
        3:'low',
        4:'close',
        5:'volume',
        6:'close_time',
        8:'number_of_trades'
    }

    df = df.drop(columns=[7,9,10,11])

    
    

    df = df.rename(columns = columns)
    
    # df = df.sort_values(by=['time'])
    df['time'] = df['time'].apply(epochToDate)
    df['close_time'] = df['close_time'].apply(epochToDate)
    
    
    return df

In [116]:
def dataPipeline(pair,interval,start_time):
    df = getKlines(pair,interval,start_time)
    df = dataCleaning(df)
    return df

## Candlestick graph

In [117]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime

def candleGraph(df):
    fig = go.Figure(data=[go.Candlestick(x=df['time'],
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'])])

    fig.show()

# candleGraph(df)

## Data exploration

In [118]:
df = dataPipeline('LUNAUSDT','15m',1646517200000)



In [120]:
df.tail()

,time,open,high,low,close,volume,close_time,number_of_trades
495,2022-03-05 03:45,83.6480,83.6990,83.1500,83.4330,165274,2022-03-05 03:59,13879
496,2022-03-05 04:00,83.4330,83.8640,83.1410,83.7370,204763,2022-03-05 04:14,14623
497,2022-03-05 04:15,83.7310,84.8660,83.6360,84.6730,395940,2022-03-05 04:29,27211
498,2022-03-05 04:30,84.6790,85.2000,84.3900,84.9860,282667,2022-03-05 04:44,19208
499,2022-03-05 04:45,84.9890,85.8060,84.9870,85.2880,437734,2022-03-05 04:59,28490


In [129]:
df = getKlines('LUNAUSDT','15m',1646519766083)
df = dataCleaning(df)

In [131]:
df.head()

,time,open,high,low,close,volume,close_time,number_of_trades
0,2022-02-28 00:00,71.5230,71.9810,71.2030,71.7020,363543,2022-02-28 00:14,24761
1,2022-02-28 00:15,71.7030,71.8010,70.7300,71.1490,492649,2022-02-28 00:29,29666
2,2022-02-28 00:30,71.1420,71.3940,70.8110,71.3840,222187,2022-02-28 00:44,16788
3,2022-02-28 00:45,71.3740,71.6070,71.0520,71.1140,180490,2022-02-28 00:59,12992
4,2022-02-28 01:00,71.1180,71.9590,71.0920,71.6710,230367,2022-02-28 01:14,16814


In [124]:
CURRENT_TIME

1646525966083